## Thread Coordination
Consider three threads running concurrently, one thread fetches data from the network, another thread displays the amount of data received so far and a third thread will process the data once the download is complete. The downloaded data is shared by all the 3 threads. We will also need 2 bools, progress flag and completed flag to cordinate the threads. The shared data and variables need to be protected with mutex's. The problem with this solution is there is too much looping with the mutex's and checking for the variables, hence it is inefficient using up more processor time. Also we need to introduce sleep in each of the thread's to give time for other threads to execute, which is again inefficient. A better solution would be for the thread to indicate to the thread system that it is waiting for something, and the thread that is doing this something can tell the thread system that it has done it, the thread system can then wakeup the waiting thread. C++ provides condition variable classes for this.

## Condition Variables
Consider two threads running concurrently, one modifies a shared string and the other uses the modified string. We can use condition varialbles to cordinate threads. Thread A tells the condiional variable that it is waiting, thread B notifies the condition variable when it updates the string, the condition variable wake's thread A up, thread A then uses the string. We use a mutex to protect critical sections, the condition variable also uses the same mutex for thread coordination. std::condition_variable has the following methods
* wait() : Takes an argument of type std::unique_lock, it unlocks its argument and blocks the thread until a notification is received. On notification, wait call returns with the mutex locked.
* wait_for() and wait_until() : Re-lock their argument if a notification is not received in time.
* notify_one() : Wakeup one of the waiting threads, scheduler decides which thread to wake up.
* notify_all() : Wake up all the waiting threads.

```
std::string sdata;
std::mutex mut;
std::condition_variable cond_var;

void reader()
{
    std::unique_lock<std::mutex> uniq_lck(mut);
    //Unlocks the mutex and waits for notification
    cond_var.wait(uniq_lck);
    
    std:::cout << "Data is \"" << sdata << "\"" << std::endl;
}

void writer()
{
    {
        std::lock_guard<std::mutex> lck_guard(mut);
        sdata = "Populated";
    }//Release the mutex
    //Notify the condition variable
    cond_var.notify_one();
}

int main()
{
    sdata = "Empty";
    std::cout << "Data is \"" << sdata << "\"" << std::endl;
    
    std::thread read(reader);
    std::thread write(writer);
    
    write.join();
    read.join();
}
```
std::condition_variable only works with std::mutex. There is aslo std::condition_variable_any, it works with any mutex object(mutex, timed_mutex etc.), including our own mutex types. It has more overhead than std::condition_variable.  
In the above example there could be a problem if we start the write thread before the read thread, the write thread could notify before the read thread starts waiting, in that case notify will be missed and the read thread will be blocked for ever, this is known as lost wakeup.  
There is also a spurious wakeup, where the writer has not called the notify but the std::condition_variable wakes up the reader anyway, this is because of the way C++ implements std::condition_variable. There is a way to solve both spurious and lost wakeups. This is to use wait() with a predicate. We use a shared bool to check if a notification is in the pipe line. The call to wait with predicate will only wait if the predicate returns false.
```
//bool flag for predicate
bool condition = false;
std::string sdata;
std::mutex mut;
std::condition_variable cond_var;

void reader()
{
    std::unique_lock<std::mutex> uniq_lck(mut);
    
    //Will wait only if condition is false
    cond_var.wait(uniq_lck, [](){return condition;});
    
    std:::cout << "Data is \"" << sdata << "\"" << std::endl;
}

void writer()
{
    {
        std::lock_guard<std::mutex> lck_guard(mut);
        sdata = "Populated";
        
        //Change the flag when still in mutex
        condition = true;
    }//Release the mutex
    //Notify the condition variable
    cond_var.notify_one();
}

int main()
{
    sdata = "Empty";
    std::cout << "Data is \"" << sdata << "\"" << std::endl;
    
    std::thread read(reader);
    std::thread write(writer);
    
    write.join();
    read.join();
}
```
Conditional variables are particularly useful in one writer and many readers case. notify_all() will wake up all the readers and they can process the data concurrently. The wakeup order is decided by the schedular.

## Futures and Promises
std::thread class does not provide a direct way to transfer data from one thread to another. So far we have used shared variable, mutex and condition varaible to communicate between the threads. This is very useful in cases like implementing a thread safe queue, however there are cases where you just want to send value from one thread to another. C++ provides std::future and std::promise classes for this. We have to use std::future and std::promise together, these setup a shared state between the threads, this shared state tranfers the data from one thread to another. Syncronization is taked care of inside the std::future and std::promise classes. Futures and promises are very useful in producer-consumer model. Producer thread will generate a result, consumer thread will wait for this result and uses it when available, reader/writer threads are an example of this model. std::promise is associated with the producer, std::future object is associated with the consumer. The consumer calls a member function of the future object, which blocks until the result becomes available. The producer sends the result through the promise object. Futures and promises also work with exceptions, the promise stores the exception, consumer catches the exception by the futures blocking function.  
std::future represents a result that is not yet available, works with many different asynchronous objects and operations, not just std::promise. A std::future object is not usually created directly, it is obtained from std::promise object or returned by an asynchronous operation. std::future is a template calss, the template parameter is the type of the data that will be stored and returned. It has get() member function, which blocks untill the result is ready, and then returns the result. There are also a wait()/wait_for()/wait_until() member functions, which block untill the operation is complete but does not return a result.  
std::promise is also a template calss, the template parameter is the type of the result. Its constructor will create the associated std::future object, we can get this by calling get_future() member function. It has set_value() member function to set and send the result. It also has set_exception() member function to set and send a std::exception.  
Parent thread creates an std::promise object, producer task function takes the promise object as an argument and calls set_value() when it has the value, the consumer task function takes the associated std::future object as argument and calls get() to wait and get the value.
```
void produce(std::promise<int> &px)
{
    try
    {
        //Produce the result
        int x = 42;
        //Code that might throw exception
        std::this_thread::sleep_for(2s);
    
        //Store/Send the result
        px.set_value(x);
    }
    catch(...)
    {
        //Store/Send the exception
        //Takes a pointer, std::current_exception() returns the current exception pointer
        px.set_exception(std::current_exception());
    }
}

void consume(std::future<int> &fx)
{
    try
    {
        //Wait and get the result
        //May throw the exception, if exception is set by the promise
        int x = fx.get();
    }
    catch(std::exception &e)
    {
        std::cout << "Exception caught : " << e.what() << std::endl;
    }
    
    std::cout << "The result is : " << x << std::endl;
}

int main()
{
    std::promise<int> prom;
    std::future<int> fut = prom.get_future();
    
    std::thread thr_produce(produce, std::ref(prom));
    std::thread thr_consumer(consume, std::ref(fut));
    
    thr_produce.join();
    thr_consume.join();
}
```
We can also directly set the exception to promise directly without the try/catch. We can use std::make_exception_ptr() to get the exception pointer from the exception object.  
In cases where there is a single producer thread and multiple consumer threads waiting for it, std::future will not work here as it is designed for a single consumer thread, the same std::future object cannot be shared between multiple threads. Also std::future is a move-only class. Instead we have to use the std::shared_future object. We can get the std::shared_future object by calling share() member on std::future object or by directly assigning get_future() member of promise to std::shared_future.
```
void produce(std::promise<int> &px)
{
    try
    {
        //Produce the result
        int x = 42;
        //Code that might throw exception
        std::this_thread::sleep_for(2s);
    
        //Store/Send the result
        px.set_value(x);
    }
    catch(...)
    {
        //Store/Send the exception
        //Takes a pointer, std::current_exception() returns the current exception pointer
        px.set_exception(std::current_exception());
    }
}

void consume(std::shared_future<int> &fx)
{
    try
    {
        //Wait and get the result
        //May throw the exception, if exception is set by the promise
        int x = fx.get();
    }
    catch(std::exception &e)
    {
        std::cout << "Exception caught : " << e.what() << std::endl;
    }
    
    std::cout << "The result is : " << x << std::endl;
}

int main()
{
    std::promise<int> prom;
    std::shared_future<int> shared_fut1 = prom.get_future();
    //Copy the shared furture obejct, they share the same state
    std::shared_future<int> shared_fut2 = shared_fut1;
    
    std::thread thr_produce(produce, std::ref(prom));
    std::thread thr_consumer1(consume, std::ref(shared_fut1));
    std::thread thr_consumer2(consume, std::ref(shared_fut2));
    
    thr_produce.join();
    thr_consumer1.join();
    thr_consumer2.join();
}
```